Notebook for plotting the results of the clustering measurements on the 4FS data. Just change the path of the cat_vars a few cells down to point to whichever set of results you wish to plot.

In [ ]:
%load_ext autoreload
%autoreload

# import modules
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import gamma as gamma_sc
from mpmath import gamma as gamma_mp

# import sys
# sys.path.append('../clustering_pipeline/')

# from imports import *
from data_io import load_object, read_to_df
from structures import catalogue_variables, results_powerspec, powerspec_measurement, CAMB_power
from aux import get_CAMB_power

In [ ]:
## global values to set e.g. font sizes and styles etc for the plots

# Defaults for font sizes, lines, etc.
rc_default = {}
rc_default['font.family'] = 'serif'
rc_default['font.size'] = 24
rc_default['axes.labelsize'] = 30
rc_default['axes.labelweight'] = 'normal'
rc_default['axes.linewidth'] = 1.0
rc_default['axes.titlesize'] = 30
rc_default['xtick.labelsize'] = 30
rc_default['ytick.labelsize'] = 30
rc_default['legend.fontsize'] = 24
# rc_default['figure.figsize'] = [400./72.27,(np.sqrt(5)-1.0)/2.0*400./72.27]
rc_default['figure.titlesize'] = 24
rc_default['savefig.dpi'] = 100
# Latex related
rc_default['text.usetex'] = True
rc_default['mathtext.fontset'] = 'custom'
rc_default['mathtext.rm'] = 'Bitstream Vera Sans'
rc_default['mathtext.it'] = 'Bitstream Vera Sans:italic'
rc_default['mathtext.bf'] = 'Bitstream Vera Sans:bold'

plt.rcParams.update(rc_default)

In [ ]:
# load the key paths / variables class object
cat_vars = load_object('catalogue_vars/2021_05_catalogue_vars')
cat_vars.__dict__

In [ ]:
# First lets have a look at what the configuration space correlation function results tables look like for one of the tracers in the output catalogues

tracer = 'LyA'

# load the pickle objects containing the results
res_wt = load_object(cat_vars.results_output + 'results_' + tracer + '_wtheta')
res_xi = load_object(cat_vars.results_output + 'results_' + tracer + '_xi')
res_wp = load_object(cat_vars.results_output + 'results_' + tracer + '_wp')

print(res_wt.__dict__.keys(), '\n') # just prints all the variables of the class

# to access the data of the object just do e.g.:
test_result = res_wt.res_avg
print(test_result, '\n')

# the below calls the print function of the class that displays the stored data via a pandas DataFrame (aside from cov matrices)
res_wt.print_results()
res_xi.print_results()
res_wp.print_results()


In [ ]:
%%time

# look at some key properties of the input and output catalogues
for tracer in cat_vars.tracers:
    target_in = cat_vars.data_input + 'input_reduced_' + tracer + '.fits'
    target_out = cat_vars.data_output + 'output_reduced_' + tracer + '.fits'
#     rand_in = read_to_df(cat_vars.randoms_input + 'table_' + tracer + '.fits')
#     rand_out = read_to_df(cat_vars.randoms_output + 'table_' + tracer + '.fits')
    df_in = read_to_df(target_in)
    df_out = read_to_df(target_out)
    N_in = len(df_in)
    N_out = len(df_out)
    n_bar = N_out / 7500
#     print(np.amin(df_out['REDSHIFT_ESTIMATE'].values), np.amax(df_out['REDSHIFT_ESTIMATE'].values))
    print('For %s there are %s input and %s output targets (%s %% completeness). Number density per sq.deg. = %s'%(tracer, N_in, N_out, round(N_out*100/N_in,1), n_bar))
#     plt.figure()
#     plt.title('%s'%(tracer))
#     plt.hist(df_out['REDDENING'].values, bins=50)
#     plt.show()
#     print(df_in.columns)
#     print(df_out.columns)
#     print(rand_in.columns)
#     print(rand_out.columns)
    

In [ ]:
import matplotlib.gridspec as gridspec
fig = plt.figure(figsize=(22, 18))

# simple power law to fit correlation functions to
def ang_power_law_fit(theta, A_w, gamma):
    return A_w * (theta)**(1-gamma)

outer = gridspec.GridSpec(2, 2, wspace=0.15, hspace=0.1)

for i in range(4):
    tracer = cat_vars.tracers[i]
    inner = gridspec.GridSpecFromSubplotSpec(2, 1,
                    subplot_spec=outer[i], wspace=0., hspace=0., height_ratios=[4,1])
    
    # load and get the data to plot
    results_file_in = cat_vars.results_input + 'results_' + tracer +'_wtheta'
    results_file_out = cat_vars.results_output + 'results_' + tracer +'_wtheta'
    res_wt_in = load_object(results_file_in)
    res_wt_out = load_object(results_file_out)
    wt_in_theta = res_wt_in.bin_mid
    wt_in_res = res_wt_in.res_avg
    wt_in_err = res_wt_in.err_avg
    wt_out_theta = res_wt_out.bin_mid
    wt_out_res = res_wt_out.res_avg
    wt_out_err = res_wt_out.err_avg
    w_fc_1 = res_wt_out.fibre_1_corr
    w_fc_2 = res_wt_out.fibre_2_corr
    
    # do a simple power law fit on the output data
    popt, pcov = curve_fit(ang_power_law_fit, wt_out_theta[wt_out_res>0], wt_out_res[wt_out_res>0], sigma=wt_out_err[wt_out_res>0])
    A_err = np.sqrt(pcov[0,0])
    pow_err = np.sqrt(pcov[1,1])
#     print("{0:.2e}, {1:.2e}, {2}, {3}".format(popt[0], A_err, popt[1], pow_err))
    
    
    for j in range(2):
        ax = plt.Subplot(fig, inner[j])
        ax.set_xscale('log')
    
        # upper plot
        if j == 0:
            ax.set_yscale('log')
            ax.set_xticks([])
            ax.set_xlim(min(wt_out_theta)*0.9, max(wt_out_theta)*1.1)
            if i in [0,2]:
                ax.set_ylabel(r'$w(\theta)$')
            
            # plot the real-splace measurements
            ax.errorbar(wt_in_theta[wt_in_res>0], wt_in_res[wt_in_res>0], yerr=wt_in_err[wt_in_res>0], capsize=3, color='red', ls='')
            ax.errorbar(wt_in_theta[wt_in_res<0], -wt_in_res[wt_in_res<0], yerr=wt_in_err[wt_in_res<0], capsize=3, color='darkred', ls='')
            ax.errorbar(wt_out_theta[wt_out_res>0], wt_out_res[wt_out_res>0], yerr=wt_out_err[wt_out_res>0], capsize=3, color='blue', ls='')
            ax.errorbar(wt_out_theta[wt_out_res<0], -wt_out_res[wt_out_res<0], yerr=wt_out_err[wt_out_res<0], capsize=3, color='darkblue', ls='')

            ax.plot(wt_in_theta, wt_in_res, marker='x', color='red', ls='--', label='%s input'%(tracer))
            ax.plot(wt_in_theta, -wt_in_res, marker='x', color='darkred', ls=':')
            ax.plot(wt_out_theta, wt_out_res, marker='o', color='blue', ls='--', label='%s output'%(tracer))
            ax.plot(wt_out_theta, -wt_out_res, marker='o', color='darkblue', ls=':')
            
            text_label = r"Power-law fit, $A_\omega$ = {0:.1e}, $\gamma$ = {1}".format(popt[0], round(popt[1],2))
            ax.plot(wt_out_theta, ang_power_law_fit(wt_out_theta, popt[0], popt[1]), color='grey', ls='--', label='Power-law fit')#text_label)
            
            
            ax.legend(fontsize=20, loc=3)
        
        # lower plot
        else:
            ax.axhline(1, c='black', ls='--')
            ax.set_xlim(min(wt_out_theta)*0.9, max(wt_out_theta)*1.1)
            ax.plot(wt_out_theta, w_fc_1, marker='o', ls='-', c='red')

            if i in [2,3]:
                ax.set_xlabel(r'$\theta$ [deg]')
            if i in [0,2]:
                ax.set_ylabel('$w_{fc}$')
            
            

        fig.add_subplot(ax)


plt.savefig(cat_vars.plots_results + 'results_wtheta_fc.pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
%%time

# for loop to plot the xi(r) results for the 4 tracers

# simple power law to fit correlation functions to
def xi_power_law_fit(r, r_0, gamma):
    return (r_0/r)**gamma

r_fit = np.zeros(4)
gamma_fit = np.zeros(4)


plt.figure(figsize=(18,13))

for i, tracer in enumerate (cat_vars.tracers):
    
    plt.subplot(2,2,i+1)
    
    if i in [0,2]:
        plt.ylabel(r'$\xi(r)$')
    if i in [2,3]:
        plt.xlabel(r'$r$ [Mpc/h]')
    plt.yscale('log')
    plt.xscale('log')
    
    # load the real-space data
    results_file_in = cat_vars.results_input + 'results_' + tracer + '_xi'
    res_xi_in = load_object(results_file_in)
    results_file_out = cat_vars.results_output + 'results_' + tracer + '_xi'
    res_xi_out = load_object(results_file_out)
    xi_in_r = res_xi_in.bin_mid
    xi_in_res = res_xi_in.res_avg
    xi_in_err = res_xi_in.err_avg
    xi_out_r = res_xi_out.bin_mid
    xi_out_res = res_xi_out.res_avg
    xi_out_err = res_xi_out.err_avg
    
    # do a simple power law fit on the output data
    if tracer in ['BG', 'LRG']:
        r_min = 3 # minimum r to be used ror fit
    else:
        r_min = 1
    popt, pcov = curve_fit(xi_power_law_fit, xi_out_r[(xi_out_res>0) & (xi_out_r>=r_min)], xi_out_res[(xi_out_res>0) & (xi_out_r>=r_min)], sigma=xi_out_err[(xi_out_res>0) & (xi_out_r>=r_min)])
    r_fit[i] = popt[0]
    gamma_fit[i] = popt[1]
    r0_err = np.sqrt(pcov[0,0])
    pow_err = np.sqrt(pcov[1,1])
#     print(r0_err, pow_err)
    A_err = np.sqrt(pcov[0,0])
    pow_err = np.sqrt(pcov[1,1])
    print("{0:.2e}, {1:.2e}, {2}, {3}".format(popt[0], A_err, popt[1], pow_err))
    
    # plot the real space data
    plt.errorbar(xi_in_r[xi_in_res>0], xi_in_res[xi_in_res>0], yerr=xi_in_err[xi_in_res>0], capsize=3, color='red', ls='')
    plt.errorbar(xi_in_r[xi_in_res<0], -xi_in_res[xi_in_res<0], yerr=xi_in_err[xi_in_res<0], capsize=3, color='darkred', ls='')
    plt.errorbar(xi_out_r[xi_out_res>0], xi_out_res[xi_out_res>0], yerr=xi_out_err[xi_out_res>0], capsize=3, color='blue', ls='')
    plt.errorbar(xi_out_r[xi_out_res<0], -xi_out_res[xi_out_res<0], yerr=xi_out_err[xi_out_res<0], capsize=3, color='darkblue', ls='')
    plt.plot(xi_in_r, xi_in_res, marker='x', color='red', ls='--', label='%s input'%(tracer))
    plt.plot(xi_in_r, -xi_in_res, marker='x', color='darkred', ls=':')
    plt.plot(xi_out_r, xi_out_res, marker='o', color='blue', ls='--', label='%s output'%(tracer))
    plt.plot(xi_out_r, -xi_out_res, marker='o', color='darkblue', ls=':')
    
    text_label = r"Power-law fit, $r_0$ = {0}, $\gamma$ = {1}".format(round(popt[0],1), round(popt[1],2))
    plt.plot(xi_out_r, xi_power_law_fit(xi_out_r, popt[0], popt[1]), color='grey', ls='--', label='Power-law fit')#text_label)
    
    
    # load and plot the redshift-space data
    if tracer in ['BG', 'LRG']:
        results_file_in = cat_vars.results_input + 'results_' + tracer + '_xi' + '_zspace'
        res_xi_in = load_object(results_file_in)
        results_file_out = cat_vars.results_output + 'results_' + tracer + '_xi' + '_zspace'
        res_xi_out = load_object(results_file_out)
        xi_in_r = res_xi_in.bin_mid
        xi_in_res = res_xi_in.res_avg
        xi_in_err = res_xi_in.err_avg
        xi_out_r = res_xi_out.bin_mid
        xi_out_res = res_xi_out.res_avg
        xi_out_err = res_xi_out.err_avg
        plt.errorbar(xi_in_r[xi_in_res>0], xi_in_res[xi_in_res>0], yerr=xi_in_err[xi_in_res>0], capsize=3, color='orange', ls='')
        plt.errorbar(xi_in_r[xi_in_res<0], -xi_in_res[xi_in_res<0], yerr=xi_in_err[xi_in_res<0], capsize=3, color='darkorange', ls='')
        plt.errorbar(xi_out_r[xi_out_res>0], xi_out_res[xi_out_res>0], yerr=xi_out_err[xi_out_res>0], capsize=3, color='green', ls='')
        plt.errorbar(xi_out_r[xi_out_res<0], -xi_out_res[xi_out_res<0], yerr=xi_out_err[xi_out_res<0], capsize=3, color='darkgreen', ls='')
        plt.plot(xi_in_r, xi_in_res, marker='x', color='orange', ls='--', label='%s input, redshift-space'%(tracer))
        plt.plot(xi_in_r, -xi_in_res, marker='x', color='darkorange', ls=':')
        plt.plot(xi_out_r, xi_out_res, marker='o', color='green', ls='--', label='%s output, redshift-space'%(tracer))
        plt.plot(xi_out_r, -xi_out_res, marker='o', color='darkgreen', ls=':')
    

    plt.legend(loc=3, fontsize=20)
    
plt.tight_layout()
# plt.savefig(cat_vars.plots_results + 'results_xi.pdf', dpi=150, bbox_inches='tight')
plt.savefig(cat_vars.plots_results + 'results_xi_new.pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
%%time

# simple power law to fit correlation functions to
def wp_power_law_fit(r_p, r_0, gamma):
    return r_p*((r_0/r_p)**gamma)*gamma_frac(gamma)

def gamma_frac(gamma):
    return (gamma_sc(1/2)*gamma_sc(0.5*(gamma-1)))/(gamma_sc(gamma/2))



# for loop to plot the w_p(r_p) results for the 4 tracers

plt.figure(figsize=(18,13))

for i, tracer in enumerate (cat_vars.tracers):
    
    results_file_in = cat_vars.results_input + 'results_' + tracer + '_wp'
    res_wp_in = load_object(results_file_in)
    results_file_out = cat_vars.results_output + 'results_' + tracer + '_wp'
    res_wp_out = load_object(results_file_out)
    
    plt.subplot(2,2,i+1)
#     plt.title('%s'%(tracer))
    if i in [0,2]:
        plt.ylabel(r'$w_p(r_p)$')
    if i in [2,3]:
        plt.xlabel(r'$r_p$ [Mpc/h]')
    plt.yscale('log')
    plt.xscale('log')
    
    wp_in_r = res_wp_in.bin_mid
    wp_in_res = res_wp_in.res_avg
    wp_in_err = res_wp_in.err_avg
    wp_out_r = res_wp_out.bin_mid
    wp_out_res = res_wp_out.res_avg
    wp_out_err = res_wp_out.err_avg
    

    popt, pcov = curve_fit(wp_power_law_fit, wp_out_r[(wp_out_res>0)], wp_out_res[(wp_out_res>0)], p0=(4, 1.2), sigma=wp_out_err[(wp_out_res>0)])
    A_err = np.sqrt(pcov[0,0])
    pow_err = np.sqrt(pcov[1,1])
    print("{0:.2e}, {1:.2e}, {2}, {3}".format(round(popt[0],3), round(A_err,3), round(popt[1],2), round(pow_err,2)))
    
    plt.errorbar(wp_in_r[wp_in_res>0], wp_in_res[wp_in_res>0], yerr=wp_in_err[wp_in_res>0], capsize=3, color='red', ls='')
    plt.errorbar(wp_in_r[wp_in_res<0], -wp_in_res[wp_in_res<0], yerr=wp_in_err[wp_in_res<0], capsize=3, color='darkred', ls='')
    plt.errorbar(wp_out_r[wp_out_res>0], wp_out_res[wp_out_res>0], yerr=wp_out_err[wp_out_res>0], capsize=3, color='blue', ls='')
    plt.errorbar(wp_out_r[wp_out_res<0], -wp_out_res[wp_out_res<0], yerr=wp_out_err[wp_out_res<0], capsize=3, color='darkblue', ls='')
    
    plt.plot(wp_in_r, wp_in_res, marker='x', color='red', ls='--', label='%s input'%(tracer))
    plt.plot(wp_in_r, -wp_in_res, marker='x', color='darkred', ls=':')
    plt.plot(wp_out_r, wp_out_res, marker='o', color='blue', ls='--', label='%s output'%(tracer))
    plt.plot(wp_out_r, -wp_out_res, marker='o', color='darkblue', ls=':')
    
#     text_label = r"Power-law fit, $r_0$ = {0}, $\gamma$ = {1}".format(round(popt[0],1), round(popt[1],2))
    text_label= "Power-law fit"
    plt.plot(wp_out_r, wp_power_law_fit(wp_out_r, popt[0], popt[1]), color='grey', ls='--', label=text_label)
#     text_label = r"Power-law fit, $r_0$ = {0}, $\gamma$ = {1}".format(round(r_fit[i],1), round(gamma_fit[i],2))
#     plt.plot(wp_out_r, wp_power_law_fit(wp_out_r, r_fit[i], gamma_fit[i]), color='grey', ls='--', label=text_label)

#     # load and plot the redshift-space data
#     if tracer in ['BG', 'LRG']:
#         results_file_in = cat_vars.results_input + 'results_' + tracer + '_wp' + '_zspace'
#         res_wp_in = load_object(results_file_in)
#         results_file_out = cat_vars.results_output + 'results_' + tracer + '_wp' + '_zspace'
#         res_wp_out = load_object(results_file_out)
#         wp_in_r = res_wp_in.bin_mid
#         wp_in_res = res_wp_in.res_avg
#         wp_in_err = res_wp_in.err_avg
#         wp_out_r = res_wp_out.bin_mid
#         wp_out_res = res_wp_out.res_avg
#         wp_out_err = res_wp_out.err_avg
#         plt.errorbar(wp_in_r[wp_in_res>0], wp_in_res[wp_in_res>0], yerr=wp_in_err[wp_in_res>0], capsize=3, color='orange', ls='')
#         plt.errorbar(wp_in_r[wp_in_res<0], -wp_in_res[wp_in_res<0], yerr=wp_in_err[wp_in_res<0], capsize=3, color='darkorange', ls='')
#         plt.errorbar(wp_out_r[wp_out_res>0], wp_out_res[wp_out_res>0], yerr=wp_out_err[wp_out_res>0], capsize=3, color='green', ls='')
#         plt.errorbar(wp_out_r[wp_out_res<0], -wp_out_res[wp_out_res<0], yerr=wp_out_err[wp_out_res<0], capsize=3, color='darkgreen', ls='')
#         plt.plot(wp_in_r, wp_in_res, marker='x', color='orange', ls='--', label='%s input, redshift-space'%(tracer))
#         plt.plot(wp_in_r, -wp_in_res, marker='x', color='darkorange', ls=':')
#         plt.plot(wp_out_r, wp_out_res, marker='o', color='green', ls='--', label='%s output, redshift-space'%(tracer))
#         plt.plot(wp_out_r, -wp_out_res, marker='o', color='darkgreen', ls=':')
    
    plt.legend(loc=3, fontsize=20)

plt.tight_layout()
# plt.savefig(cat_vars.plots_results + 'results_wp.pdf', dpi=150, bbox_inches='tight')
plt.savefig(cat_vars.plots_results + 'results_wp_new.pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# look at the correlation matrix, calculated from covariance

catalogue = 'output' # ['input' or 'output']
statistic = 'wtheta' # ['wtheta', 'xi', or 'wp']

def correlation_from_covariance(covariance):
    v = np.sqrt(np.diag(covariance))
    outer_v = np.outer(v, v)
    correlation = covariance / outer_v
    correlation[covariance == 0] = 0
    return correlation



if statistic == 'wtheta':
    label1 = r'$\theta_i$ [deg]'
    label2 = r'$\theta_j$ [deg]'
if statistic == 'xi':
    label1 = r'$r_i$ [Mpc/h]'
    label2 = r'$r_j$ [Mpc/h]'
if statistic == 'wp':
    label1 = r'$r_{p, i}$ [Mpc/h]'
    label2 = r'$r_{p, j}$ [Mpc/h]'

plt.figure(figsize=(14,12))

for i, tracer in enumerate(cat_vars.tracers):
    plt.subplot(2,2,i+1)
    plt.title('%s - %s'%(tracer, catalogue))
    plt.xlabel(label1)
    plt.ylabel(label2)


    # load covariance and get correlation matrix, then plot
    path = ''
    if catalogue == 'input':
        path += cat_vars.results_input + 'results_' + tracer + '_'
    else:
        path += cat_vars.results_output + 'results_' + tracer + '_'

    res = load_object(path + statistic)
    corr_m = correlation_from_covariance(res.cov_avg)

    plt.imshow(np.flip(corr_m, axis=0), cmap=plt.get_cmap('RdBu'), vmin=-1, vmax=1)
    plt.xticks(np.linspace(0,19,20), np.round(res.bin_mid,2), rotation='vertical', size=13)
    plt.yticks(np.linspace(0,19,20), np.flip(np.round(res.bin_mid,2)), size=13)
    
    plt.colorbar()


plt.tight_layout()
plt.savefig(cat_vars.plots_results + 'corr_matrix_' + catalogue + '_' + statistic + '.pdf', dpi=150, bbox_inches='tight')
plt.show()



In [ ]:
# plot the pair counts for output catalogues for w(theta) measurement

plt.figure(figsize=(18,13))

for i, tracer in enumerate(cat_vars.tracers):
    
    # load the results file
    res = load_object(cat_vars.results_output + 'results_' + tracer + '_wtheta')
    
    plt.subplot(2,2,i+1)
    plt.yscale('log')
    plt.xscale('log')
    plt.title('Output %s'%(tracer))
    plt.xlabel(r'$\theta$ [deg]')
    plt.ylabel('Pair counts')
    
    plt.plot(res.bin_mid, res.DD1, c = 'green', ls='--', label='DD field 1')
    plt.plot(res.bin_mid, res.DD2, c = 'green', ls='-.', label='DD field 2')
    
    plt.plot(res.bin_mid, res.RR1, c = 'blue', ls='--', label='RR field 1')
    plt.plot(res.bin_mid, res.RR2, c = 'blue', ls='-.', label='RR field 2')
    
    plt.plot(res.bin_mid, res.DR1, c = 'red', ls='--', label='DR field 1')
    plt.plot(res.bin_mid, res.DR2, c = 'red', ls='-.', label='DR field 2')
    
    plt.legend()
    plt.tight_layout()
    
plt.show()  

# Now for some power spectrum plots

In [ ]:
%%time

CAMB_0 = get_CAMB_power(redshift=0, cosmo=cat_vars.cosmo, camb_h=cat_vars.hubble_h)

# for loop to plot the P_l(k) results for the 4 tracers

Nmesh = 1024
plot_zspace = True
window='tsc'   # 'cic' or 'tsc'
custom_name = ''

plt.figure(figsize=(18,13))

for i, tracer in enumerate (cat_vars.tracers):
    
    plt.subplot(2,2,i+1)
    plt.yscale('log')
    if i > 1:
        plt.xlabel('$k$ [$h$Mpc$^{-1}$]')      
    if i % 2 == 0:
        plt.ylabel('$P_{\ell}(k)$ [$h^{-3}$Mpc$^3$]')
    #plt.title(r'Window conv. $P^{\rm Gal}_{\ell}(k)$ - %s'%(tracer))
    
    
    
        
    # load and plot the real space measurements
    results_file_in = cat_vars.results_input + 'results_' + tracer + '_pk_' + str(Nmesh) + '_' + window + custom_name
    res_pk_in = load_object(results_file_in)
    results_file_out = cat_vars.results_output + 'results_' + tracer + '_pk_' + str(Nmesh) + '_' + window + custom_name
    res_pk_out = load_object(results_file_out)
    
    bin0_in = res_pk_in.bin0
    bin0_out = res_pk_out.bin0
    
    plt.errorbar(bin0_in.bin_mid, bin0_in.Pk0, yerr=bin0_in.Pk0_err, capsize=3, color='red', marker='x', label='$P_0(k)$, %s input'%(tracer))
    plt.errorbar(bin0_out.bin_mid, bin0_out.Pk0, yerr=bin0_out.Pk0_err, capsize=3, color='blue', marker='o', label='$P_0(k)$, %s output'%(tracer))
    
    # set limits
    plt.xlim(np.amin(bin0_in.bin_mid)*0.5,np.amax(bin0_in.bin_mid)*1.05)
    plt.ylim(np.amin(bin0_out.Pk0)*0.5, np.amax(bin0_out.Pk0)*2)
    
    
    # plot redshift space
    if plot_zspace==True:
        if tracer in ['BG', 'LRG']:
            results_file_in = cat_vars.results_input + 'results_' + tracer + '_pk_' + str(Nmesh) + '_' + window + '_zspace' + custom_name
            res_pk_in = load_object(results_file_in)
            results_file_out = cat_vars.results_output + 'results_' + tracer + '_pk_' + str(Nmesh) + '_' + window + '_zspace' + custom_name
            res_pk_out = load_object(results_file_out)
            bin0_in = res_pk_in.bin0
            bin0_out = res_pk_out.bin0
            plt.errorbar(bin0_in.bin_mid, bin0_in.Pk0, yerr=bin0_in.Pk0_err, capsize=3, color='orange', marker='x', ls='--', label='$P^S_0(k)$, %s input'%(tracer))
            plt.errorbar(bin0_out.bin_mid, bin0_out.Pk0, yerr=bin0_out.Pk0_err, capsize=3, color='green', marker='o', ls='--', label='$P^S_0(k)$, %s output'%(tracer))
    
    
    
    # plot camb line
    plt.plot(CAMB_0.k_range, CAMB_0.nonlin_power, c='black', ls='--', label=r'CAMB matter $P_\mathrm{NL}(k)$ at $z=0$')

    
    plt.legend(loc=1, fontsize=20)

plt.tight_layout()
plt.savefig(cat_vars.plots_results + 'results_Pk.pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
catalogue = 'output'
Nmesh = 1024
window = 'tsc'
real_or_redshift = 'real'
custom_name = ''

# camb_z = 0
# CAMB_0 = get_CAMB_power(redshift=camb_z, cosmo=cat_vars.cosmo, camb_h=cat_vars.hubble_h)

plt.figure(figsize=(25,8))

plt.subplot(1,2,1)
tracer = 'LRG'
plt.yscale('log')
plt.xlabel('$k$ [$h$Mpc$^{-1}$]')
plt.ylabel('$P_{\ell}(k)$ [$h^{-3}$Mpc$^3$]')
# plt.title('Window convolved galaxy $P(k)$ plot - %s %s'%(catalogue, tracer))
plt.title('%s %s'%(tracer, catalogue))

# load results and pull a few bin params
result_in = load_object(cat_vars.results_input + 'results_' + tracer + '_pk_' + str(Nmesh) + '_' + window + '_zbins' + custom_name)
bin0 = result_in.bin0
bin2 = result_in.bin2
bin4 = result_in.bin4

plt.xlim(np.amin(bin0.bin_mid)*0.2,np.amax(bin0.bin_mid)*1.03)
plt.ylim(1e1, 6e4)

plt.errorbar(bin0.bin_mid, bin0.Pk0, yerr=bin0.Pk0_err, capsize=3, marker='x', color='blue', label='$P_0(k)$, %s $\leq z \leq$ %s'%(round(bin0.Z_min, 2),round(bin0.Z_max, 2)))
plt.errorbar(bin2.bin_mid, bin2.Pk0, yerr=bin2.Pk0_err, capsize=3, marker='o', color='purple', label='$P_0(k)$, %s $\leq z \leq$ %s'%(round(bin2.Z_min, 2),round(bin2.Z_max, 2)))
plt.errorbar(bin4.bin_mid, bin4.Pk0, yerr=bin4.Pk0_err, capsize=3, marker='s', color='red', label='$P_0(k)$, %s $\leq z \leq$ %s'%(round(bin4.Z_min, 2),round(bin4.Z_max, 2)))

plt.plot(CAMB_0.k_range, CAMB_0.nonlin_power, c='black', ls='--', label=r'CAMB matter $P_\mathrm{NL}(k)$ at $z=%s$'%(camb_z))

plt.legend(fontsize=20)


plt.subplot(1,2,2)
tracer = 'QSO'
plt.yscale('log')
plt.xlabel('$k$ [$h$Mpc$^{-1}$]')
# plt.ylabel('$P_{\ell}(k)$ [$h^{-3}$Mpc$^3$]')
# plt.title('Window convolved galaxy $P(k)$ plot - %s %s'%(catalogue, tracer))
plt.title('%s %s'%(tracer, catalogue))

# load results and pull a few bin params
result_in = load_object(cat_vars.results_input + 'results_' + tracer + '_pk_' + str(Nmesh) + '_' + window + '_zbins' + custom_name)
bin0 = result_in.bin0
bin2 = result_in.bin2
bin4 = result_in.bin4

plt.xlim(np.amin(bin0.bin_mid)*0.2,np.amax(bin0.bin_mid)*1.03)
plt.ylim(1e2, 4e4)

plt.errorbar(bin0.bin_mid, bin0.Pk0, yerr=bin0.Pk0_err, capsize=3, marker='x', color='blue', label='$P_0(k)$, %s $\leq z \leq$ %s'%(round(bin0.Z_min, 2),round(bin0.Z_max, 2)))
plt.errorbar(bin2.bin_mid, bin2.Pk0, yerr=bin2.Pk0_err, capsize=3, marker='o', color='purple', label='$P_0(k)$, %s $\leq z \leq$ %s'%(round(bin2.Z_min, 2),round(bin2.Z_max, 2)))
plt.errorbar(bin4.bin_mid, bin4.Pk0, yerr=bin4.Pk0_err, capsize=3, marker='s', color='red', label='$P_0(k)$, %s $\leq z \leq$ %s'%(round(bin4.Z_min, 2),round(bin4.Z_max, 2)))

plt.plot(CAMB_0.k_range, CAMB_0.nonlin_power, c='black', ls='--', label=r'CAMB matter $P_\mathrm{NL}(k)$ at $z=%s$'%(camb_z))

plt.legend(fontsize=20)

plt.savefig(cat_vars.plots_results + 'results_Pk_zbins.pdf', dpi=150, bbox_inches='tight')

plt.show()